<a href="https://colab.research.google.com/github/MSimonFRA-UAS/JuniorUni/blob/main/BigramSprachModell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from typing import List
import numpy as np
from collections import Counter
import random

In [15]:
class BigramSprachModell(object):
    """
    Eine Klasse, welche ein Bigram Sprachmodell implementiert

    Attribute:
        bigram_anz: Eine Liste von Dictionaries. Das i-te Dictionary enthaelt die Haeufigkeiten fuer Worte, welche auf das i-te Wort folgen.
        D.h., falls an der Stelle i = 42 das Wort "und" gespeichert wird, speichert bigram_anz[42] ein Dictionary von Haeufigkeiten, wie etwa {dann: 125, danach: 23. ...}
        fuer alle Worte, die je nach "und" auftreten.
        vok_anz: Eine Liste von Haeufigkeiten jedes Worts im Vokabular.
    """
    def __init__(self, bigram_anz: dict, vok_anz: dict):
        self.bigram_anz = bigram_anz
        self.vok_anz = vok_anz

    def get_vokabular(self):
        """
        :Ausgabe: Das Vokabular des Trainingstexts
        """
        return self.vok_anz.keys()

    def get_wkt(self, vorh_wort: str, wort: str) -> float:
        """
        Berechnet die Wahrscheinlichkeit P(naechstes Wort = wort | vorheriges Wort = vorh_wort)
        :Parameter vorh_wort: vorheriges Wort
        :Parameter wort: das naechste Wort
        :Ausgabe: Die bedingte Wahrscheinlichkeit
        """
        anz_nach_vorh_wort = self.bigram_anz[vorh_wort]
        if wort in anz_nach_vorh_wort:
            naechstes_wort_in_kontext_anz = anz_nach_vorh_wort[wort]
        else:
            naechstes_wort_in_kontext_anz = 0
        return naechstes_wort_in_kontext_anz / self.vok_anz[vorh_wort]


def trainiere_bigram_modell(train_seqs: List[List[str]]) -> BigramSprachModell:
    bigram_anz = {}
    vok_anz = {}
    vorh_wort_anz = {}
    for train_seq in train_seqs:
        for i in range(1, len(train_seq)):
            vorh_wort = train_seq[i-1]
            wort = train_seq[i]
            if vorh_wort not in bigram_anz:
                bigram_anz[vorh_wort] = {}
            if wort in bigram_anz[vorh_wort]:
                bigram_anz[vorh_wort][wort] += 1
            else:
                bigram_anz[vorh_wort][wort] = 1
            if wort not in vok_anz:
                vok_anz[wort] = 1
            else:
                vok_anz[wort] += 1
    return BigramSprachModell(bigram_anz, vok_anz), bigram_anz, vok_anz

In [16]:
BEGIN_SYMBOL = "<S>"
END_SYMBOL = "</S>"


def read_wikitext(path: str) -> List[List[str]]:
    """
    Liest a Wikitext file at the given path.
    :Parameter path: Pfad des einzulesenden Datensatzes
    :Ausgabe: Eine geschachtelte Liste List[List[str]]: Die erste Liste ist eine Liste von Zeilen, die zweite Liste ist eine Liste der Worte (Strings) in dieser Zeile.
    """
    print("Trainingsdaten werden eingelesen aus der Datei " + path)
    f = open(path)
    lines = []
    for line in f:
        if len(line.strip()) > 0:
            this_line = [BEGIN_SYMBOL]
            split_line = line.split(" ")
            for word in split_line:
                if len(word.strip()) > 0:
                    this_line.append(word.strip())
            this_line.append(END_SYMBOL)
            lines.append(this_line)
    print("%i Zeilen wurden eingelesen." % len(lines))
    return lines

In [17]:
train = read_wikitext("wiki.valid.tokens")

Trainingsdaten werden eingelesen aus der Datei wiki.valid.tokens
4883 Zeilen wurden eingelesen.


In [18]:
train[-1]

['<S>',
 'In',
 'der',
 'Generaldebatte',
 'über',
 'die',
 'Politik',
 'seiner',
 'Regierung',
 'kündigte',
 'Mas',
 'am',
 '25.',
 'September',
 '2012',
 'im',
 'Regionalparlament',
 'an',
 ',',
 'Neuwahlen',
 'für',
 'den',
 '25.',
 'November',
 '2012',
 'anzuberaumen',
 '.',
 '</S>']

In [19]:
lm, bigram_anzahl, vok_anzahl = trainiere_bigram_modell(train)

In [20]:
bigram_anzahl['November']

{'2009': 3,
 '1854': 1,
 '1933': 1,
 '2016': 1,
 '1875': 1,
 '1992': 1,
 'wurde': 1,
 '1962': 3,
 '1955': 1,
 '1846': 1,
 '1968': 1,
 '2011': 2,
 '2013': 1,
 '1706': 1,
 '1827': 1,
 '1997': 1,
 'statt': 1,
 '1918': 2,
 '1936': 1,
 'und': 1,
 '1998': 2,
 '1924': 1,
 '2006': 1,
 '1904': 1,
 '1848': 1,
 '1914': 1,
 '2015': 1,
 '1912': 1,
 '2010': 2,
 '1980': 2,
 '1981': 1,
 'nahezu': 1,
 '1438': 1,
 '2000': 1,
 '1919': 1,
 '2017': 1,
 '1946': 1,
 '1872': 1,
 '1944': 2,
 '2012': 3,
 '2004': 1,
 '2005': 1,
 '1925': 1}

In [21]:
vok_anzahl['November']

55

In [22]:
def vorhersage(lm, kontext_wort: str, anz_worte = 5):
    """
    Gibt top anz_worte naechsten Worte nach dem kontext_wort aus
    :Parameter lm: Sprachmodell
    :Parameter kontext_wort: Kontextwort
    :Parameter anz_worte: Laenge der Liste mit Worten mit den hoechsten Wahrscheinlichkeiten
    """
    counter = Counter()
    for wort in lm.get_vokabular():
        counter[wort] = lm.get_wkt(kontext_wort, wort)
    result = ""
    for (wort, count) in counter.most_common(anz_worte):
        result += "(" + wort + ", " + repr(counter[wort]) + "), "
    print("Top 5 Worte und Wahrscheinlichkeiten fuer Wort nach \"" + kontext_wort + "\": " + result[:-2])

In [23]:
vorhersage(lm, "November")

Top 5 Worte und Wahrscheinlichkeiten fuer Wort nach "November": (2009, 0.05454545454545454), (1962, 0.05454545454545454), (2012, 0.05454545454545454), (2011, 0.03636363636363636), (2010, 0.03636363636363636)


In [24]:
def sample_wort(lm, kontext_wort: str):
    """
    :Parameter lm: Sprachmodell
    :Parameter kontext_wort: Kontextwort
    :Ausgabe: Ein zufaellig gesampletes Wort, welches dem Kontextwort folgt entsprechend den Wahrscheinlichkeiten von lm.get_wkt.
    """


In [ ]:
def sample_satz(lm, kontext_wort: str):
    """
    :Parameter lm: Sprachmodell
    :Parameter kontext_wort: Kontextwort
    :Ausgabe: Satz mit bis zu 10 Worten durch sukzessive Anwendung der sample_wort Routine
    """


In [ ]:
 print(repr(sample_satz(lm, "November")))